In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!rm -rf /root/.pytorch_pretrained_bert/imdb_fine/5

!mkdir /root/.pytorch_pretrained_bert
!mkdir /root/.pytorch_pretrained_bert/imdb_fine

!pip3 install pytorch_pretrained_bert

import nltk
nltk.download('stopwords')

!cp -r -f /content/drive/'My Drive'/project_q/content/

In [ ]:
import pandas as pd
import sys

sys.path.append('/content/project_q/')

from bert_cls import *
from model_fitetune import *
from model_aug import *
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm_notebook
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
dataset_name = "/content/project_q/leading_sent_big.txt"
finetune_dataset = "/content/project_q/imdb_labelled.txt"

In [ ]:
finetune_bert = FinetuneBert(ds_path=finetune_dataset, model_name='imdb_fine', num_epochs=4)
finetune_bert.finetune()

In [ ]:
sent_data = pd.read_csv(dataset_name, 
                        sep=';', 
                        names=['label','sent','sentence'], header=0, encoding = "utf-8")

In [ ]:
X = sent_data.sentence.values
xu = [s.encode('utf-8') for s in X]
x = np.array([bytes([i for i in j if i< 128]).decode('utf-8') for j in xu])
y = np.array(sent_data.label.values)
lbl = np.array(sent_data.sent.values)

d_lbl = {}
for i in range(len(x)):
    d_lbl[x[i]] = lbl[i]

In [ ]:
po, pa, t, test_folds, train_folds = get_metrics_aug(x, y, n_folds=13, temp=1.2, random_state=64, use_untuned=False, if_lbl=True, use_stop = True,
                                                     lr = 2e-5, n_random=4, n_epochs=4, n_samples=4, n_rounds=3, d_lbl = d_lbl)

In [ ]:
a = accurancy_score(po, t)
b = accurancy_score(pa, t)
print(sum(a)/len(a))
print(sum(b)/len(b))

drow_plots(a,b)

In [ ]:
import scipy.stats
scipy.stats.wilcoxon(a,b, alternative='less')

In [ ]:
##augmentation demonstration

In [ ]:
s = 'Would you vote for a woman for President if she were qualified in every other aspect?'
aug_bert.aug_sent(s, label = 1, temp=1, n_rounds=1, n_samples=1, n_random=2)

In [ ]:
##cls demonstration

In [ ]:
aug_bert = BertAug(model_name='imdb_fine/5', use_untuned=use_untuned, use_stop = use_stop)

model = BertClassificationModel(n_epochs=n_epochs, lr=lr)
model.fit(X,y)
predict = model.predict_batch_proba(X_test)
